In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib ipympl

import random
from PIL import Image, ImageDraw
import cv2
import numpy as np

from tifffile import imsave
from tqdm import tqdm

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from csbdeep.utils import Path, normalize

np.random.seed(42)
lbl_cmap = random_label_cmap()

# REFERENCE IMAGES FROM VIDEOS

In [ ]:
def grayscale_crop_and_resize(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	npImage = npImage[y1:y2, x1:x2]
	return cv2.resize(npImage, (500, 500))

def get_frame(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	alpha = Image.new('L', (w, h), 0)
	draw = ImageDraw.Draw(alpha)
	draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
	npAlpha = np.array(alpha)
	npImage = npImage*npAlpha
	ind = np.where(npImage == 0)
	npImage[ind] = npImage[200, 200]
	npImage = npImage[y1:y2, x1:x2]
	return cv2.resize(npImage, (500, 500))

In [ ]:
save_path = './training_dataset/'
nDrops = 50

source_path_25b25r = "./data/25b25r-1.mp4" 
xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r = 95, 30, 535, 470
video_25b25r = cv2.VideoCapture(source_path_25b25r)
video_25b25r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FPS))
n_frames_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"25b25r video has {n_frames_25b25r} frames with a resolution of {w_25b25r}x{h_25b25r} and a framerate of {fps_25b25r} fps")


source_path_49b1r  = "./data/49b1r.mp4"
xmin_49b1r,  ymin_49b1r,  xmax_49b1r,  ymax_49b1r  = 20, 50, 900, 930
nDrops_post_merge_49b1r = 49
video_49b1r = cv2.VideoCapture(source_path_49b1r)
video_49b1r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FPS))
n_frames_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_COUNT))
print(f"49b1r video has {n_frames_49b1r} frames with a resolution of {w_49b1r}x{h_49b1r} and a framerate of {fps_49b1r} fps")
print(f"25b25r video can be cropped to : {ymax_25b25r - ymin_25b25r, xmax_25b25r - xmin_25b25r}")
print(f"49b1r  video can be cropped to : {ymax_49b1r -  ymin_49b1r,  xmax_49b1r -  xmin_49b1r }")

start_49b1r = get_frame(video_49b1r, 0, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
end_49b1r = get_frame(video_49b1r, n_frames_49b1r-1, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
start_25b25r = get_frame(video_25b25r, 0, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)
end_25b25r = get_frame(video_25b25r, n_frames_25b25r-1, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)

fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0,0].imshow(start_49b1r, cmap='gray', vmin=0, vmax=255)
ax[0,0].set_title('Start 49b1r')
ax[0,1].imshow(end_49b1r, cmap='gray', vmin=0, vmax=255)
ax[0,1].set_title('End 49b1r')
ax[1,0].imshow(start_25b25r, cmap='gray', vmin=0, vmax=255)
ax[1,0].set_title('Start 25b25r')
ax[1,1].imshow(end_25b25r, cmap='gray', vmin=0, vmax=255)
ax[1,1].set_title('End 25b25r')
plt.tight_layout()
plt.show()
n_samples_49b1r  = 100
n_samples_25b25r = 100
n_samples = n_samples_49b1r + n_samples_25b25r

np.random.seed(0)
sample_frames_49b1r  = np.sort(np.array(random.sample(range(30000 ), n_samples_49b1r )), axis=0)
sample_frames_25b25r = np.sort(np.array(random.sample(range(n_frames_25b25r), n_samples_25b25r)), axis=0)

for frame in tqdm(sample_frames_49b1r):
    imsave(f"{save_path}49b1r/image/49b1r_frame{frame}.tif", get_frame(video_49b1r, frame, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r))

for frame in tqdm(sample_frames_25b25r):
    imsave(f"{save_path}25b25r/image/25b25r_frame{frame}.tif", get_frame(video_25b25r, frame, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r))

# SYNTHETIC DATASET GENERATOR

In [ ]:
def get_frame(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
	alpha = Image.new('L', (w, h), 0)
	draw = ImageDraw.Draw(alpha)
	draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
	npAlpha = np.array(alpha)
	npImage = npImage*npAlpha
	ind = np.where(npImage == 0)
	npImage[ind] = npImage[200, 200]
	npImage = npImage[y1:y2, x1:x2]
	npImage = cv2.resize(npImage, (500, 500))
	return npImage

In [ ]:
source_path_25b25r = "./data/25b25r-1.mp4" 
xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r = 95, 30, 535, 470
video_25b25r = cv2.VideoCapture(source_path_25b25r)
video_25b25r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FPS))
n_frames_25b25r = int(video_25b25r.get(cv2.CAP_PROP_FRAME_COUNT))


source_path_49b1r  = "./data/49b1r.mp4"
xmin_49b1r,  ymin_49b1r,  xmax_49b1r,  ymax_49b1r  = 20, 50, 900, 930
video_49b1r = cv2.VideoCapture(source_path_49b1r)
video_49b1r.set(cv2.CAP_PROP_POS_FRAMES, 0)
w_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_WIDTH))
h_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FPS))
n_frames_49b1r = int(video_49b1r.get(cv2.CAP_PROP_FRAME_COUNT))

start_49b1r = get_frame(video_49b1r, 0, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
end_49b1r = get_frame(video_49b1r, n_frames_49b1r-1, xmin_49b1r, ymin_49b1r, xmax_49b1r, ymax_49b1r, w_49b1r, h_49b1r)
start_25b25r = get_frame(video_25b25r, 0, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)
end_25b25r = get_frame(video_25b25r, n_frames_25b25r-1, xmin_25b25r, ymin_25b25r, xmax_25b25r, ymax_25b25r, w_25b25r, h_25b25r)

fig, ax = plt.subplots(2, 2, figsize=(10, 8))
ax[0,0].imshow(start_49b1r, cmap='gray', vmin=0, vmax=255)
ax[0,0].set_title('Start 49b1r')
ax[0,1].imshow(end_49b1r, cmap='gray', vmin=0, vmax=255)
ax[0,1].set_title('End 49b1r')
ax[1,0].imshow(start_25b25r, cmap='gray', vmin=0, vmax=255)
ax[1,0].set_title('Start 25b25r')
ax[1,1].imshow(end_25b25r, cmap='gray', vmin=0, vmax=255)
ax[1,1].set_title('End 25b25r')
plt.tight_layout()
plt.show()

In [ ]:
def draw_circle_with_index(mask, center, radius, index):
    cv2.circle(mask, center, radius, (index), -1)

def overlap_between_circles(existing_circles, center, radius, min_distance):
    for existing_center in existing_circles:
        distance = np.linalg.norm(np.array(existing_center[0]) - np.array(center))
        if distance < radius + existing_center[1]:
            return True
    return False

def create_gaussian(center, img_width, img_height, sigma, ampl):
    center_x, center_y = center
    x = np.linspace(0, img_width-1, img_width)
    y = np.linspace(0, img_height-1, img_height)
    X, Y = np.meshgrid(x, y)
    gaussian = np.exp(-((X-center_x)**2 + (Y-center_y)**2) / (2.0 * sigma**2))
    return np.round(ampl*(gaussian / np.max(gaussian))).astype(np.uint8)

def generate_synthetic_image():
    num_circles = 50 

    # Create a black background image
    height, width = 500, 500
    image = np.random.randint(65, 75, (height, width), dtype=np.uint8) #np.ones((height, width), dtype=np.uint8)*70
    outer_radius = 200
    mask = np.zeros((height, width), dtype=np.uint8)
    list_of_centers = []
    list_of_distances = []
    circles_array = np.zeros((height, width), dtype=np.uint8)

    # Draw circles on the background image and mask
    for i in range(num_circles):
        while True:
            # Generate random circle properties
            theta = random.uniform(0, 2 * np.pi)
            r = random.randint(0, outer_radius)
            center = (int(width/2 + r * np.cos(theta)), int(height/2 + r * np.sin(theta)))

            feature_radius = 10# random.randint(9, 10)

            if not overlap_between_circles(list_of_centers, center, feature_radius, feature_radius):
                list_of_centers.append([center, feature_radius])
                break
        color = 110 #(random.randint(0, 255))
        index = i + 1  # Assign unique index (starting from 1)
        # Draw the circle on the image 
        # lineType = 4 for 4-connected line, 8 for 8-connected line, LINE_AA for antialiased line
        cv2.circle(image, center, feature_radius, color, -1, lineType=4) 

        # draw circles as gaussian distribution
        circles_array += create_gaussian(center, width, height, feature_radius/2, 10)

        # Draw the circle with its index on the mask
        draw_circle_with_index(mask, center, feature_radius, index)

    # Draw the outer circle mimicking the petri dish
    image = image + circles_array
    cv2.circle(image, (int(height/2), int(width/2)), int(width/2), 150) 
    cv2.circle(image, (int(height/2), int(width/2)), int(width/2)-4, 150) 
    kernel = np.array([[0, -1, 0],
                    [-1, 5,-1],
                    [0, -1, 0]])
    #image = cv2.filter2D(image, ddepth=-1, kernel=kernel)
    image = cv2.GaussianBlur(image, (5, 5), 4)

    return image, mask

In [ ]:
test_img, test_mask = generate_synthetic_image()
fig, axs = plt.subplots(2, 3, figsize=(10, 5), sharex=True, sharey=True)
axs[0,0].imshow(test_img, cmap="gray", vmin=0, vmax=255)
axs[0,1].imshow(start_49b1r, cmap='gray', vmin=0, vmax=255)
axs[0,2].imshow(start_25b25r, cmap='gray', vmin=0, vmax=255)
axs[1,0].imshow(test_mask, cmap=lbl_cmap)
axs[1,1].imshow(end_49b1r, cmap='gray', vmin=0, vmax=255)
axs[1,2].imshow(end_25b25r, cmap='gray', vmin=0, vmax=255)

axs[0,0].set(xticks=[], yticks=[], title='Synthetic')
axs[0,1].set(xticks=[], yticks=[], title='49b1r')
axs[0,2].set(xticks=[], yticks=[], title='25b25r')
axs[1,0].set(xticks=[], yticks=[], title='Synthetic mask')
axs[1,1].set(xticks=[], yticks=[], title='49b1r')
axs[1,2].set(xticks=[], yticks=[], title='25b25r')
plt.tight_layout()
plt.show()

In [ ]:
for i in tqdm(range(1000)):
    img, mask = generate_synthetic_image()
    imsave(f'./synthetic_dataset/image/synthetic_image_{i}.tif', img)
    imsave(f'./synthetic_dataset/mask/synthetic_mask_{i}.tif', mask)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(img_aug, cmap='gray', vmin=0, vmax=255)
ax[0].set_title('Augmented Image')
ax[1].imshow(lbl_aug, cmap='gray')
ax[1].set_title('Augmented Mask')
plt.tight_layout()
plt.show()

# HOUGH-CIRCLE DATASET GENERATOR

## FROM DATA OF 49b-1r HOUGH CIRCLE METHOD DETECTION

In [ ]:
def get_frame(cap, frame, x1, y1, x2, y2, w, h, preprocess):
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
    ret, image = cap.read()
    if preprocess:
        npImage = np.array(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY))
        alpha = Image.new('L', (w, h), 0)
        draw = ImageDraw.Draw(alpha)
        draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
        npAlpha = np.array(alpha)
        npImage = npImage*npAlpha
        ind = np.where(npImage == 0)
        npImage[ind] = npImage[200, 200]
        npImage = npImage[y1:y2, x1:x2]
        return npImage
    elif not preprocess:
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        raise ValueError("preprocess must be a boolean")

In [ ]:
trajectories = pd.read_parquet('./49b_1r/old/tracking_data/hough/tracking_hough_trackpy_linking.parquet')
# rename column to be consistent with the other datasets
trajectories.rename(columns={'d':'r'}, inplace=True)
# remove rows with NaN values in the radius column
trajectories.dropna(subset=['r'], inplace=True)
display(trajectories.head())

In [ ]:
video = cv2.VideoCapture('/Users/matteoscandola/MasterThesis/tracking/data/49b1r.mp4')
video.set(cv2.CAP_PROP_POS_FRAMES, 0)
_, first_frame = video.read()

In [ ]:
# sample x frames from the hough-method tracking dataset
n_samples = 1000
np.random.seed(42)
sample_frames = np.sort(np.random.choice(trajectories.frame.unique().astype(int), size=n_samples, replace=False))

for frame in tqdm(sample_frames):
    frame_img = get_frame(video, frame, 55, 55, 880, 880, 920, 960, True)
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/image/49b1r_frame{frame}.tif", frame_img)
    df = trajectories.loc[(trajectories.frame == frame), ["x", "y", "r"]]
    image = np.zeros((880 - 55, 880 - 55), dtype=np.uint8)
    count = 0
    for _, droplet in df.iterrows():
        x, y, radius = droplet['x'], droplet['y'], droplet['r']
        cv2.circle(image, (int(x)- 55, int(y) - 55), int(radius), int(count), thickness=-1)
        count += 1
    imsave(f"/Users/matteoscandola/MasterThesis/tracking/train_49b-1r/mask/49b1r_frame{frame}.tif", image)    

## GENERATING NEW HOUGH CIRCLE DETECTIONS

In [ ]:
def get_frame_hough(cap, frame, x1, y1, x2, y2, w, h):
	cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
	ret, image = cap.read()
	npImage = np.array(image)
	alpha = Image.new('L', (w, h), 0)
	draw = ImageDraw.Draw(alpha)
	draw.pieslice(((x1, y1), (x2, y2)), 0, 360, fill=255)
	npAlpha = np.array(alpha)
	npImage = cv2.cvtColor(npImage, cv2.COLOR_BGR2GRAY)*npAlpha 
	ind = np.where(npImage == 0)
	npImage[ind] = npImage[200, 200]
	npImage = npImage[y1:y2, x1:x2]
	return npImage 
	
def hough_loc_frame(correct_n, frame, img, parameters):
	found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **parameters)
	if found_circles is not None:
		return np.hstack((found_circles[0], (np.ones((found_circles.shape[1], 1), dtype=int)*frame),\
							np.ones((found_circles.shape[1], 1), dtype=int)*found_circles.shape[1]))
	else:
		return np.hstack((np.ones((1, 3))*-1, np.array([[frame, 0]])))
		
def hough_feature_location(sample_frames, correct_n, params):
	temp = []
	for frame in tqdm(sample_frames):
		img = get_frame_hough(video, frame, xmin, ymin, xmax, ymax, w, h)
		temp.append(hough_loc_frame(correct_n, frame, img, params))
	
	temp_df = pd.DataFrame(np.concatenate([arr for arr in temp]), columns = ["x", "y", "d", "frame", "nDroplets"])
	temp_df["frame"] = temp_df["frame"].astype(int)
	temp_df["nDroplets"] = temp_df["nDroplets"].astype(int)
	err_frames = temp_df.loc[temp_df.nDroplets != correct_n].frame.unique().astype(int)
	loss = err_frames.shape[0]/sample_frames.shape[0]
	return temp_df, err_frames, loss

def optimize_params(x, *args):
	frames, correct_n = args
	params = {"dp":x[0], "minDist":int(x[1]), "param1":x[2], "param2":x[3], "minRadius":int(x[4]), "maxRadius":int(x[5])}
	errs = 0
	for i in tqdm(frames):
		img = get_frame_hough(video, i, xmin, ymin, xmax, ymax, w, h)
		found_circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT_ALT, **params)
		if (found_circles is not None) and (found_circles.shape[1] == correct_n):
			pass
		else:
			errs += 1
			
	loss = errs/frames.shape[0]
	a = [loss, x[0], int(x[1]), x[2], x[3], int(x[4]), int(x[5])]
	print(a)
	return loss

In [ ]:
n_samples = 100
random.seed(0)
sample_frames = np.sort(np.array(random.sample(range(n_frames), n_samples)), axis=0)

test_params = {"dp":1.5, "minDist":10, "param1":20, "param2":0.7, "minRadius":5, "maxRadius":30}

temp_df, err_frames, loss = hough_feature_location(sample_frames, nDrops, test_params)
print(loss, err_frames)
temp_df_test = temp_df.loc[temp_df.frame == err_frames[1]]
test_img = get_frame_hough(video, err_frames[1], xmin, ymin, xmax, ymax, w, h)
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
ax.imshow(test_img, cmap="gray")
for i in range(temp_df_test.nDroplets.iloc[0]):
    circle = plt.Circle((temp_df_test.x.iloc[i], temp_df_test.y.iloc[i]), temp_df_test.d.iloc[i], color='r', fill=True, alpha=0.5)
    ax.add_artist(circle)
ax.set(xticks=[], yticks=[], title=f"Frame {err_frames[1]} -- {temp_df_test.nDroplets.iloc[0]} droplets")
plt.show()

In [ ]:
# paramters of HoughCircles --> dp, minDist, param1, param2, minRadius, maxRadius
init_guess =  [2, 5, 20, 0.7, 7, 12] # initial guess for the parameters
params_bounds = [(1, 3), (1, 10), (1, 100), (0.3, 1), (5, 10), (10, 15)] # bounds for the parameters

opt_result = dual_annealing(optimize_params, x0 = init_guess, args = (sample_frames, nDrops), bounds = params_bounds, maxiter = 2000)